This notebook is thought to be executed in Google colab, but ignoring this first cell can be executed in any jupyter environment.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/automl/'
!apt-get install -y libhdf5-serial-dev
!apt-get install -y build-essential swig
!pip install tables
!pip install deap update_checker tqdm stopit
!pip install xgboost
!pip install tpot
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn

!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip install autokeras
!pip install xtoy
!pip install scipy
!pip install numpy

!apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o




Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libhdf5-serial-dev is already the newest version (1.10.0-patch1+docs-4).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   194  100   194    0     0    979      0 --:--:-- --:--:-- --:--:--   979
     |████████████████████████████████| 6.7MB 3.4MB/s 
ERROR: xtoy 0.5.63 has requirement scikit-learn<0.20,>=0.18.1, but you'll have

  Using cached https://files.pythonhosted.org/packages/0d/3a/b92670f5c368c20329ecc4c255993fae7934564d485c3ed7ea7b8da7f741/scikit_learn-0.20.2-cp36-cp36m-manylinux1_x86_64.whl
ERROR: xtoy 0.5.63 has requirement scikit-learn<0.20,>=0.18.1, but you'll have scikit-learn 0.20.2 which is incompatible.
ERROR: xtoy 0.5.63 has requirement xgboost==0.72.1, but you'll have xgboost 0.90 which is incompatible.
ERROR: auto-sklearn 0.5.2 has requirement scikit-learn<0.20,>=0.19, but you'll have scikit-learn 0.20.2 which is incompatible.
  Found existing installation: scikit-learn 0.19.2
    Uninstalling scikit-learn-0.19.2:
      Successfully uninstalled scikit-learn-0.19.2


  Using cached https://files.pythonhosted.org/packages/06/7a/442f7da21792566012e5c7e5a7dffa44c1b6cc05c0c27856bbc8a7718b28/xgboost-0.72.1-py2.py3-none-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f9/c8/8db4108aba5e2166cd2ea4eafa1a4b82f89240a1fa85733029cc2358ad1f/scikit_learn-0.19.2-cp36-cp36m-manylinux1_x86_64.whl
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.
ERROR: autokeras 0.4.0 has requirement scikit-learn==0.20.2, but you'll have scikit-learn 0.19.2 which is incompatible.
ERROR: auto-sklearn 0.5.2 has requirement xgboost>=0.80, but you'll have xgboost 0.72.1 which is incompatible.
  Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90
  Found existing installation: scikit-learn 0.20.2
    Uni

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html


In [0]:
import pandas as pd
import seaborn as sns
sns.set(style="ticks")
sns.set_palette("husl")
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tpot import TPOTClassifier
import autosklearn.classification
warnings.filterwarnings('ignore')
%matplotlib inline
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from xtoy.toys import Toy

h2o.init()

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpqm05hywt
  JVM stdout: /tmp/tmpqm05hywt/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpqm05hywt/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,4 days
H2O cluster name:,H2O_from_python_unknownUser_z7nmer
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


Loading some helper functions and data

In [0]:
oo_jaen_df = pd.read_csv(f'{root_path}/oo_jaen.csv')
X_train, X_test, y_train, y_test = train_test_split(oo_jaen_df.drop(columns='increase'), oo_jaen_df['increase'], 
                                                    train_size=0.75, test_size=0.25)

In [0]:
tpot_model = TPOTClassifier(generations=50, population_size=100, verbosity=2, n_jobs=-1)
tpot_model.fit(X_train, y_train)
print('TPOT accuracy', tpot_model.score(X_test, y_test))

Generation 1 - Current best internal CV score: 0.5794945594945594
Generation 2 - Current best internal CV score: 0.5794945594945594
Generation 3 - Current best internal CV score: 0.5838497718497718
Generation 4 - Current best internal CV score: 0.5838497718497718
Generation 5 - Current best internal CV score: 0.5865885105885107
Generation 6 - Current best internal CV score: 0.5865885105885107
Generation 7 - Current best internal CV score: 0.5865885105885107
Generation 8 - Current best internal CV score: 0.5891859131859132
Generation 9 - Current best internal CV score: 0.5891859131859132
Generation 10 - Current best internal CV score: 0.5943807183807184
Generation 11 - Current best internal CV score: 0.5943807183807184
Generation 12 - Current best internal CV score: 0.596978120978121
Generation 13 - Current best internal CV score: 0.596978120978121
Generation 14 - Current best internal CV score: 0.596978120978121
Generation 15 - Current best internal CV score: 0.596978120978121
Generati

In [0]:
autosklearn_model = autosklearn.classification.AutoSklearnClassifier(time_left_for_this_task=60*60*2)
autosklearn_model.fit(X_train, y_train)
print('Autosklearn accuracy', autosklearn_model.score(X_test, y_test))

[WARNING] [2019-10-04 21:35:46,910:EnsembleBuilder(1):b3ece38fac9ea9f13908345bcfdff990] No models better than random - using Dummy Score!
[WARNING] [2019-10-04 21:35:46,933:EnsembleBuilder(1):b3ece38fac9ea9f13908345bcfdff990] No models better than random - using Dummy Score!
[WARNING] [2019-10-04 21:54:59,054:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-10-04 21:54:59,054:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
1
['/tmp/autosklearn_tmp_124_643/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_124_643/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_124_643/.auto-sklearn/ensembles/1.0000000002.ensemble', '/tmp/autosklearn_tmp_124_643/.auto-sklearn/ensembles/1.0000000003.ensemble', '/tmp/autosklearn_tmp_124_643/.auto-sklearn/ensembles/1.0000000004.ensemble', '/tmp/autosklearn_tmp

In [0]:
toy = Toy()
toy.fit(X_train, y_train)
print('toy accuracy', toy.score(X_test, y_test))

make_scorer(f1_weighted_score)
estimator: ridge
unique_combinations 16
Types [2] and maxint [15] detected
--- Evolve in 16 possible combinations ---
gen	nevals	avg     	min     	max     
0  	5     	0.431542	0.411139	0.456431
1  	3     	0.456682	0.4398  	0.478969
2  	2     	0.455216	0.42672 	0.478969
3  	4     	0.439002	0.417631	0.478969
4  	2     	0.4553  	0.408256	0.478969
5  	5     	0.43625 	0.408886	0.462424
6  	3     	0.448767	0.418428	0.474709
7  	4     	0.452359	0.432649	0.484603
8  	4     	0.442988	0.403289	0.484603
9  	2     	0.470857	0.435667	0.484603
10 	3     	0.451047	0.402902	0.484603
(0.44048495964648593,)
cbp {'estimator__alpha': 100000}
Best individual is: {'estimator__alpha': 100000}
with fitness: 0.44048495964648593
estimator: rf
unique_combinations 216
Types [1, 1, 1, 1, 1, 1] and maxint [5, 3, 2, 2, 0, 0] detected
--- Evolve in 216 possible combinations ---
gen	nevals	avg     	min     	max     
0  	5     	0.559178	0.527397	0.585083
1  	2     	0.543889	0.438915	0.585

In [0]:
train_full_df = X_train.copy()
train_full_df['increase'] = y_train
test_full_df = X_test.copy()
test_full_df['increase'] = y_test
train = h2o.H2OFrame(train_full_df)
test = h2o.H2OFrame(test_full_df)
x = train.columns
y = "increase"
x.remove(y)

train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

h2o_model = H2OAutoML(max_runtime_secs = 60*60*2)
h2o_model.train(x = x, y = y, training_frame = train)
predictions = h2o_model.leader.predict(test).as_data_frame()['predict']
print(h2o_model.leaderboard)
print(h2o_model.leader)
print('H2O accuracy', accuracy_score(predictions, y_test))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
deeplearning prediction progress: |███████████████████████████████████████| 100%


model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid_1_AutoML_20191006_165247_model_33,0.608557,1.28828,0.576311,0.332134
DeepLearning_grid_1_AutoML_20191006_165247_model_5,0.61125,1.17982,0.575021,0.330649
DeepLearning_grid_1_AutoML_20191006_165247_model_31,0.61225,1.19658,0.574259,0.329774
DeepLearning_grid_1_AutoML_20191006_165247_model_11,0.614737,1.17475,0.5947,0.353669
DeepLearning_grid_1_AutoML_20191006_165247_model_32,0.615129,1.43991,0.585307,0.342584
DeepLearning_grid_1_AutoML_20191006_165247_model_4,0.620159,1.35126,0.583922,0.340965
DeepLearning_grid_1_AutoML_20191006_165247_model_27,0.621056,0.974153,0.553749,0.306638
DeepLearning_grid_1_AutoML_20191006_165247_model_9,0.62215,1.07336,0.559731,0.313299
DeepLearning_grid_1_AutoML_20191006_165247_model_25,0.622253,1.00438,0.549487,0.301936
DeepLearning_grid_1_AutoML_20191006_165247_model_34,0.622543,0.843039,0.510272,0.260377



Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid_1_AutoML_20191006_165247_model_33


Status of Neuron Layers: predicting increase, 3-class classification, multinomial distribution, CrossEntropy loss, 514,003 weights/biases, 5.9 MB, 39,948 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,22,Input,15,,,,,,,,,
1,,2,500,RectifierDropout,0,0,0,0.00378558,0.00196425,0,0.000356973,0.0779273,0.478262,0.0257313
2,,3,500,RectifierDropout,0,0,0,0.00596644,0.0118627,0,-0.00725735,0.046601,0.982783,0.00536533
3,,4,500,RectifierDropout,0,0,0,0.0683825,0.150096,0,-0.00142569,0.0469906,0.736144,0.375612
4,,5,3,Softmax,,0,0,0.0364945,0.183973,0,-0.0653205,0.251198,-0.0671879,0.0767513




ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.15003889500032203
RMSE: 0.3873485445955903
LogLoss: 0.538510663481039
Mean Per-Class Error: 0.13677538772190592

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


...


Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_classification_error
0,,2019-10-06 18:00:37,0.000 sec,None,0.000000,0,0.0,NaN,NaN,NaN,NaN
1,,2019-10-06 18:00:42,13 min 39.556 sec,219 obs/sec,2.844737,1,1081.0,0.543571,0.930791,0.702437,0.436842
2,,2019-10-06 18:00:47,13 min 44.695 sec,328 obs/sec,8.492105,3,3227.0,0.553021,0.930788,0.692001,0.423684
3,,2019-10-06 18:00:53,13 min 50.734 sec,414 obs/sec,16.973684,6,6450.0,0.554885,0.954966,0.689921,0.444737
4,,2019-10-06 18:00:59,13 min 56.590 sec,458 obs/sec,25.523684,9,9699.0,0.475525,0.694214,0.772274,0.328947
5,,2019-10-06 18:01:05,14 min 2.382 sec,485 obs/sec,34.052632,12,12940.0,0.482778,0.779905,0.765274,0.297368
6,,2019-10-06 18:01:10,14 min 7.770 sec,508 obs/sec,42.581579,15,16181.0,0.404709,0.526811,0.835050,0.236842
7,,2019-10-06 18:01:16,14 min 13.503 sec,521 obs/sec,51.115789,18,19424.0,0.453086,0.702589,0.793258,0.268421
8,,2019-10-06 18:01:22,14 min 19.404 sec,528 obs/sec,59.644737,21,22665.0,0.413300,0.584711,0.827972,0.236842
9,,2019-10-06 18:01:28,14 min 25.169 sec,535 obs/sec,68.205263,24,25918.0,0.401573,0.525971,0.837597,0.213158



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,prec,1.000000,1.000000,0.055219
1,Unnamed: 0,0.973456,0.973456,0.053753
2,prec-4w_sum,0.947891,0.947891,0.052342
3,price_perc-2w,0.924059,0.924059,0.051026
4,prec-52w_sum,0.897869,0.897869,0.049579
5,freeze_damage,0.878331,0.878331,0.048501
6,racha,0.865275,0.865275,0.047780
7,price_perc-4w,0.859609,0.859609,0.047467
8,price_perc-8w,0.856957,0.856957,0.047320
9,production_tons-12m_sum,0.837936,0.837936,0.046270



See the whole table with table.as_data_frame()

H2O accuracy 0.5511811023622047


In [0]:
h2o_model.leaderboard

model_id,mean_per_class_error,logloss,rmse,mse
DeepLearning_grid_1_AutoML_20191006_165247_model_33,0.608557,1.28828,0.576311,0.332134
DeepLearning_grid_1_AutoML_20191006_165247_model_5,0.61125,1.17982,0.575021,0.330649
DeepLearning_grid_1_AutoML_20191006_165247_model_31,0.61225,1.19658,0.574259,0.329774
DeepLearning_grid_1_AutoML_20191006_165247_model_11,0.614737,1.17475,0.5947,0.353669
DeepLearning_grid_1_AutoML_20191006_165247_model_32,0.615129,1.43991,0.585307,0.342584
DeepLearning_grid_1_AutoML_20191006_165247_model_4,0.620159,1.35126,0.583922,0.340965
DeepLearning_grid_1_AutoML_20191006_165247_model_27,0.621056,0.974153,0.553749,0.306638
DeepLearning_grid_1_AutoML_20191006_165247_model_9,0.62215,1.07336,0.559731,0.313299
DeepLearning_grid_1_AutoML_20191006_165247_model_25,0.622253,1.00438,0.549487,0.301936
DeepLearning_grid_1_AutoML_20191006_165247_model_34,0.622543,0.843039,0.510272,0.260377
